## Setup - SQL Query Helper

Run this cell first to set up the helper function for executing SQL queries.

In [ ]:
import subprocess
import os

def run_sql(query, description="Query"):
    """
    Execute SQL query using Docker container with pymssql.
    
    Args:
        query: SQL query string
        description: Optional description of the query
    """
    # Create temporary SQL file
    with open('/tmp/query.sql', 'w') as f:
        f.write(query)
    
    # Create Python script to execute the query
    script = """import pymssql
import sys

try:
    conn = pymssql.connect(
        server='sqlserver',
        port=1433,
        user='sa',
        password='YourStrong@Passw0rd',
        database='university'
    )
    cursor = conn.cursor()
    
    with open('/query.sql', 'r') as f:
        sql = f.read()
    
    cursor.execute(sql)
    
    # Get column names
    columns = [desc[0] for desc in cursor.description] if cursor.description else []
    
    # Fetch results
    rows = cursor.fetchall()
    
    # Print results
    if columns:
        # Print header
        print(" | ".join(str(col) for col in columns))
        print("-" * (sum(len(str(col)) for col in columns) + len(columns) * 3 - 1))
         
    if rows:
        for row in rows:
            print(" | ".join(str(val) if val is not None else "NULL" for val in row))
        print(f"\\n({len(rows)} row(s) returned)")
    else:
        print("(0 rows returned)")
    
    cursor.close()
    conn.close()
    
except Exception as e:
    print(f"Error: {e}", file=sys.stderr)
    sys.exit(1)
"""
    
    with open('/tmp/run_query.py', 'w') as f:
        f.write(script)
    
    # Run query in Docker container
    print(f"\n=== {description} ===")
    result = subprocess.run([
        'docker', 'run', '--rm',
        '--network', 'sql-data-analysis_db_network_uek',
        '-v', '/tmp/query.sql:/query.sql',
        '-v', '/tmp/run_query.py:/run_query.py',
        'sql-data-analysis-init-db',
        'python', '/run_query.py'
    ], capture_output=True, text=True)
    
    # Print output
    if result.stdout:
        print(result.stdout, end='')
    
    if result.returncode != 0:
        if result.stderr:
            print("STDERR:", result.stderr)
    
    # Cleanup
    try:
        os.remove('/tmp/query.sql')
        os.remove('/tmp/run_query.py')
    except:
        pass
    
    return None  # Explicitly return None to avoid Jupyter displaying return value

print("SQL helper function loaded successfully!")
print("Use run_sql(query, description) to execute queries.")


SQL helper function loaded successfully!
Use run_sql(query, description) to execute queries.


## How to Use

For each exercise below:
1. Write your SQL query as a string
2. Use `run_sql(your_query, "Exercise X.XX")` to execute it
3. Check the results against the expected output

**Example:**
```python
query = """
SELECT first_name, surname 
FROM students 
WHERE group_no = 'DMIe1001'
"""
run_sql(query, "Students in DMIe1001")
```

## Exercise 1.01
The number of students in the database.

**Expected result:** 35

In [10]:
query = """
SELECT count(*) from students
"""
run_sql(query)


=== Query ===
35
35


## Exercise 1.02
The number of students assigned to any group.

**Expected result:** 28

In [ ]:
-- Write your SQL query here

## Exercise 1.03
The number of students who are not assigned to any group.

**Expected result:** 7

In [ ]:
-- Write your SQL query here

## Exercise 1.04
The number of groups to which at least one student is assigned.

**Expected result:** 12

In [ ]:
-- Write your SQL query here

## Exercise 1.05
The names of groups to which at least one student is assigned, along with the number of assigned students. The query should also return information on how many students are not assigned to any group. The column returning the number of students should be named `no_of_students`. Data sorted in ascending order by number of students.

**Expected result:** 13 records
- There is one student in five groups
- Two students in four groups
- Four in one group, five in one group, six in one group, and seven in one group

In [ ]:
-- Write your SQL query here

## Exercise 1.06
Names of groups with at least three students enrolled, along with the number of students. The column returning the number of students should be named `no_of_students`. Data sorted in ascending order by number of students.

**Expected result:** 3 records
- ZMIe2011: 4 students
- ZZIe2002: 5 students
- DMIe1001: 6 students

In [ ]:
-- Write your SQL query here

## Exercise 1.07
All possible grades and how many times each grade was awarded (the column should be named `no_of_grades`). Data sorted by grades.

**Expected result:** 8 records
- Grade 2.0 was awarded 13 times
- Grade 5.5 four times
- Grade 6.0 has not been awarded

In [ ]:
-- Write your SQL query here

## Exercise 1.08
The names of all departments and the total number of module hours taught by the staff employed in these departments. The column showing the number of hours should be named `total_hours`. Data sorted in ascending order by the total_hours column.

**Expected result:** 11 records
- For first six records total_hours is NULL
- Last record: Department of Informatics, 117 hours

In [ ]:
-- Write your SQL query here

## Exercise 1.09
The name of each lecturer along with the number of modules they give (the query should also return the names of lecturers who do not give any modules). The column containing the number of modules should be named `no_of_modules`. Data sorted in descending order by surname.

**Expected result:** 28 records
- First: Wright, doesn't give any module
- Third: White, gives one module

In [ ]:
-- Write your SQL query here

## Exercise 1.10
Surnames and first names of lecturers who give at least two modules, together with the number of modules they give. Data sorted in descending order by number of modules and then in ascending order by surname.

**Expected result:** 6 records
- First: Harry Jones, 4 modules
- Last: Lily Taylor, 2 modules

In [ ]:
-- Write your SQL query here

## Exercise 1.11
The surnames and first names of all students named Bowen who received at least one grade, along with their average grade (for each Bowen separately). The column returning the average should be named `avg_grade`. The data should be sorted in descending order by surname and in descending order by first name.

**Expected result:** Two records
- Harry Bowen, average 3.7
- Charlie Bowen, average 2.0

In [ ]:
-- Write your SQL query here

## Exercise 1.12
The surnames and first names of lecturers who give at least one module, together with the average grades they have given to students (if a lecturer has not yet given any grades, they should also appear on the list). The column showing the average should be named `avg_grade`. Data sorted in descending order by average.

**Expected result:** 11 records
- First: James Robinson, average 5.0
- One lecturer did not give any grades

In [ ]:
-- Write your SQL query here

## Exercise 1.12b
The surnames and first names of all lecturers, together with the average grades they have given to students (if a lecturer has not yet given any grades or does not teach any modules, they should also appear on the list). The column showing the average should be named `avg_grade`. Data sorted in descending order by average.

**Expected result:** 28 records
- In 18 cases the average is NULL (lecturers who do not give any modules or give modules but have not yet awarded any grades)

In [ ]:
-- Write your SQL query here

## Exercise 1.13a
The names of modules and the amount that the university must prepare for payments to employees teaching Statistics and Economics (separately). If there are multiple modules named Statistics or Economics, the sum for them is to be calculated together. The query should therefore return two records (one for Statistics modules, the other for Economics modules).

The amount for one module should be calculated as the product of the hourly rate of the lecturer and the number of hours allocated to the lecture.

**Expected result:** One record: Economics 1200.00

**Question:** Why did the query not return data about the Statistics modules?

In [ ]:
-- Write your SQL query here

## Exercise 1.13b
Query returning a single number: the amount that the university must prepare for payments for all modules given. The amount for one module should be calculated as the product of the hourly rate of the lecturer and the number of hours allocated to that lecture. Remember to name the column returning the searched amount.

**Expected result:** 20265.00

**Question:** Is it possible to calculate the full amount for all the modules given? Justify your answer.

In [ ]:
-- Write your SQL query here

## Exercise 1.13c
The amount that the university must prepare for payments for modules to which no lecturer is assigned, assuming that the average value from the overtime_rate field in the acad_positions table should be paid for each hour of such modules.

**Expected result:** 7649.99

In [ ]:
-- Write your SQL query here

## Exercise 1.13d
The maximum amount that the university must set aside for payments for modules for which this amount cannot be calculated. These are modules to which no lecturer has been assigned, or to which a lecturer has been assigned but whose academic degree/title is unknown.

**Expected result:** 13200

In [ ]:
-- Write your SQL query here

## Exercise 1.14
The surnames and first names of lecturers, together with the total number of hours of modules given by each of them individually, but only if the total number of hours of modules exceeds 30. The column showing the number of hours should be named `no_of_hours`. Data sorted in descending order by number of hours.

**Expected result:** 5 records
- First: Jones Harry, 72 hours
- Last: Katie Davies, 55 hours

In [ ]:
-- Write your SQL query here

## Exercise 1.15
The names of all groups and the number of students in each group (the column should be named `no_of_students`). Data sorted in ascending order by number of students and then by group number.

**Expected result:** 23 records
- There are no students in 11 groups
- Last record: group DMIe1011, 6 students

In [ ]:
-- Write your SQL query here

## Exercise 1.16
The names of all modules whose names begin with the letter A and the average grades for all these modules separately (if there are many such modules, the average should be calculated for each of them separately). If there is no grade for a given module, it should also appear on the list. The column should be named `average`.

**Expected result:** 3 records
- Advanced databases: NULL
- Advanced statistics: 4.25
- Ancient history: 4.25

In [ ]:
-- Write your SQL query here

## Exercise 1.17
The names of those departments in which at least two doctors work, together with the number of doctors working in those departments (the last column should be named `no_of_doctors`). The data should be sorted in descending order by the number of doctors and in ascending order by department name.

**Expected result:** 3 records
- Department of Informatics: three doctors
- Department of History and Department of Mathematics: two doctors each

In [ ]:
-- Write your SQL query here

## Exercise 1.18
The surnames, first names and department names of the three lecturers who gave the most modules, together with the number of modules given by each of them. The column showing the number of modules should be named `no_of_modules`. If the third person on the list gives the same number of modules as the subsequent ones, these subsequent ones should also appear in the query result table.

**Expected result:** 6 records
- Harry Jones gives 4 modules
- Thomas Evans gives 3 modules
- Four lecturers give 2 modules each

In [ ]:
-- Write your SQL query here

## Exercise 1.19
Identifiers, module names, and names of departments responsible for conducting modules for which it is not possible to determine the amount to be paid for their delivery, together with the first and last names of any of the employees of those departments. Data sorted by module_id.

**Hint:** First, create a calculated field that combines the surname and first name into a single field. Use the CONCAT function or the + operator. Remember to insert a space between the surname and first name.

**Expected result:** 6 records
- Module IDs: 4, 7, 13, 15, 20, 22

In [ ]:
-- Write your SQL query here